In [1]:
import os
import re
import time
import shutil
import pandas as pd
import numpy as np
from openai import OpenAI

In [2]:

# Your api key if using cilent.
client = OpenAI(api_key="Your_api_key", base_url="https://api.deepseek.com")
# Prompt folder.
str_resource = "dataset\\RQ3_dataset\\prompt\\"
# Choice folder.
str_choice = "dataset\\RQ3_dataset\\choice\\"
# Selection index result will be saved in result_file_dire.
result_file_dire = "RQ3_result.csv"
# If you need to save the textual process, set this path.
str_result_txt = "dataset\\RQ3_dataset\\result_txt\\"

In [3]:

def call_LLM(string, print = False):
    # Example as deepseek; You can also using other API or local LLM.
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": string},
        ],
        max_tokens=50,
        stream=False
    )
    if print == True:
        print(response.choices[0].message.content)
    
    return response.choices[0].message.content


In [5]:
# By calling this function, we can get the result of LLM.
def test_LLM(message):
    # We add the tails to prompt, so that it will be more easier to get the result of LLM.
    message = message + "\nYour answer:\n###\n"
    str_return = call_LLM(message, print = False)
    #str_return = str_return[str_return.index("###") + 3:]
    return str_return

In [6]:
def compare(str_return, choice_list):
    # Compare result of LLM and the choice_list and ground truth; return the index of selected choice.
    # find_id is the return value; 0 means fail to select any widget.
    find_id = 0
    # find_pos means the match position in the return string of LLM, aim at to find the first one matched. 
    find_pos = len(str_return) + 1
    # We try to match the first occurrence of a valid widget name
    ad = 1
    for choice in choice_list:
        choice = choice[choice.find(":") + 2:]
        now_pos = str_return.find(choice)
        if now_pos >= 0 and now_pos < find_pos:
            find_id = ad
            find_pos = now_pos
        ad = ad + 1
    # If find_id is not zero, return the value.
    if find_id != 0:
        return find_id
    # Else we try to match the index.
    for i in range(len(choice_list)):
        str_com = str(i + 1)
        now_pos = str_return.find(choice)
        if now_pos >= 0 and now_pos < find_pos:
            find_id = ad
            find_pos = now_pos
        ad = ad + 1
    # Whether success or not, return the find_id.
    return find_id

In [7]:
result_file = open(result_file_dire, "w", encoding = "UTF-8")
# repeat 5 times for one prompt.
result_file.write("Name,1,2,3,4,5")

files_resource = os.listdir(str_resource)
for file_resource in files_resource:
    result_file.write("\n")
    # Write the filename in the first column to the result_file.
    result_file.write(file_resource)
    resource_dire = str_resource + file_resource
    
    # Get the main string of this page.
    file_list = file_resource.split("_")
    file_index1 = file_list[-1].replace(".txt", "")
    file_list.pop()
    file_index2 = file_list[-1]
    file_list.pop()
    file_name = ""
    for file_str in file_list:
        file_name = file_name + file_str + "_"
    file_name = file_name.replace("dataset", "")

    # Get choice list.
    choice_dire = str_choice + "choice" + file_name + ".txt"
    # For information absence page (page index 1 to 15), use special choice list.
    if int(file_index2) > 0 and int(file_index2) <= 15:
        choice_dire = str_choice + "choice" + file_name + file_index2 + ".txt"
    choice_list = []
    with open(choice_dire, 'r') as file_choice: 
        line = file_choice.readline() 
        while line: 
            line = line.replace("\n", "")
            choice_list.append(line)
            line = file_choice.readline()
    
    resource_str = open(resource_dire, 'r').read()
    print(file_resource + " text begin,", end = " ")
    for i in range(0, 5):
        # Get LLM result and match result.
        result_str = test_LLM(resource_str)
        comp_res = compare(result_str, choice_list)
        
        # Save result string.
        result_txt_dire = str_result_txt + "GPT_answer" + file_name + str(file_index2) + "_" + str(file_index1) + ".txt"
        with open(result_txt_dire, "a+", encoding = "UTF-8") as f:
            f.write("###Answer repeat time = " + str(i + 1) + "\n")
            f.write(result_str)
            f.write("\n###Answer end\n")

        # Write to result csv.
        result_file.write(",")
        result_file.write(str(comp_res))
        time.sleep(1)
        print(str(i + 1) + " ", end = " ")
    print("end")
result_file.close()

datasetchrono_1_0_0.txt text begin, 1  2  3  4  5  end
datasetchrono_1_0_1.txt text begin, 1  2  3  4  5  end
datasetchrono_1_0_2.txt text begin, 1  2  3  4  5  end
datasetchrono_1_0_3.txt text begin, 1  2  3  4  5  end
datasetchrono_1_0_4.txt text begin, 1  2  3  4  5  end
datasetchrono_1_0_5.txt text begin, 1  2  3  4  5  end
datasetchrono_1_0_6.txt text begin, 1  2  3  4  5  end
datasetchrono_1_0_7.txt text begin, 1  2  3  4  5  end
datasetchrono_1_0_8.txt text begin, 1  2  3  4  5  end
datasetchrono_1_0_9.txt text begin, 1  2  3  4  5  end
datasetchrono_1_11_0.txt text begin, 1  2  3  4  5  end
datasetchrono_1_11_1.txt text begin, 1  2  3  4  5  end
datasetchrono_1_11_2.txt text begin, 1  2  3  4  5  end
datasetchrono_1_11_3.txt text begin, 1  2  3  4  5  end
datasetchrono_1_11_4.txt text begin, 1  2  3  4  5  end
datasetchrono_1_11_5.txt text begin, 1  2  3  4  5  end
datasetchrono_1_11_6.txt text begin, 1  2  3  4  5  end
datasetchrono_1_11_7.txt text begin, 1  2  3  4  5  end
da